In [ ]:
import json
import logging
import os
import shutil
from datetime import datetime
from pathlib import Path
import pytz

from databasetools import NotionBlock
from databasetools import NotionClient
from databasetools import NotionDatabase
from databasetools import NotionExporter
from databasetools import NotionPage
from databasetools.adapters.notion import utils

import os

env_file = Path(Path.cwd()).parent / ".env"
print(f"Looking for environment variables in {env_file}")
if env_file.exists():
    from dotenv import load_dotenv

    load_dotenv(env_file)
    print(f"Loaded environment variables from {env_file}")

IMPORTANT_LIST = [
    "MONGO_URI",
    "NOTION_API_KEY",
    "CONFLUENCE_SPACE",
    "CONFLUENCE_SPACE_KEY",
    "CONFLUENCE_API_KEY",
    "TEST_DATABASE_ID",
    "PAGE_ID",
    "CHATDB_URL",
    "STAFF_URL",
    "COMPONENTS_URL",
    "DATABASE_URL",
    "PAGE_URL",
    "CHATGPT_USER_ID",
    "CHATGPT_NOTION_DATABASE_URL",
    "CHATGPT_NOTION_DATABASE_ID",
]
for k in os.environ.keys():
    if k in IMPORTANT_LIST:
        print(f"{k}: {os.environ[k]}")

NOTION_API_KEY = os.getenv("NOTION_API_KEY")
PAGE_URL = os.getenv("PAGE_URL")
TEST_DATABASE_ID = os.getenv("TEST_DATABASE_ID")
PAGE_ID = os.getenv("PAGE_ID")

In [ ]:
from typing import List

notion_client = NotionClient(token=NOTION_API_KEY)
page_payload = {
    "filter": {
        "value": "page",
        "property": "object",
    },
    "sort": {
        "direction": "descending",
        "timestamp": "last_edited_time",
    },
}

page_payload["query"] = "Channel Controller"

results = notion_client.client.search(**page_payload).get("results")
pages : List[NotionPage] = []
for result in results:
    pages.append(NotionPage(token=NOTION_API_KEY,
        page_id=result["id"]))



In [ ]:
database_payload = {
    "filter": {
        "value": "database",
        "property": "object",
    },
    "sort": {
        "direction": "descending",
        "timestamp": "last_edited_time",
    },
}
from pprint import pprint
# database_payload["query"] = ""
results = notion_client.client.search(**database_payload).get("results")
# pages : List[NotionPage] = []
property_set = set()
# database_ids = []
database_type_map = {}
for result in results:
    # pprint(result)
    properties = result["properties"]
    # database_ids.append(result["id"])
    database_type_map[result["id"]] = properties
    for key in properties.keys():
        property_set.add(key)
    # pages.append(NotionPage(token=NOTION_API_KEY,
    #     page_id=result["id"]))

In [ ]:
from pprint import pprint
client = NotionClient(token=NOTION_API_KEY)

# Test get_metadata
INTEGRATION_URL = "https://www.notion.so/cosgroma/Integration-Test-Area-456ea602e2754100a30f952b42908650?pvs=4"
integration_page_id = utils.extract_id(INTEGRATION_URL)
integration_metadata = client.get_metadata(page_id=integration_page_id)

pprint(integration_metadata)

# Test get_recent_pages
# recent_pages = client.get_recent_pages()

# Test get_blocks
# blocks = client.get_blocks(block_id=PAGE_ID)
# for page in recent_pages:
#     print(page)

# Test get_database
# database = client.get_database(database_id=TEST_DATABASE_ID)

Filters can be chained with the and and or keys so that multiple filters are applied at the same time. (See Query a database for additional examples.)


```json
{
  "and": [
    {
      "property": "Done",
      "checkbox": {
        "equals": true
      }
    }, 
    {
      "or": [
        {
          "property": "Tags",
          "contains": "A"
        },
        {
          "property": "Tags",
          "contains": "B"
        }
      ]
    }
  ]
}
```
JQL Form
```sql
Done = true AND (Tags = 'A' OR Tags = 'B')
```

Compact Notation
```bash
condition_query="Done && (Tags contains 'A' || Tags contains 'B')"
```